In [ ]:
!pip install tqdm

En primer lugar, se han descargado los ficheros fits del sector 68 usando los links del fichero .sh descargado de la web de MAST correspondiente a las LC (curvas de luz) del sector 68.

In [ ]:
import os
import requests
from tqdm import tqdm
import re
from google.colab import drive

drive.mount('/content/drive')
base_path = '/content/drive/My Drive/TFM_b'
lc_dir = os.path.join(base_path, 'light_curves_s68')
os.makedirs(lc_dir, exist_ok=True)

sh_file_path = os.path.join(base_path, 'tesscurl_sector_68_lc.sh')

with open(sh_file_path, 'r') as file:
    lines = file.readlines()

urls = [line.split()[-1] for line in lines if line.startswith('curl')]

# limpio las rutas para evitar errores
def clean_filename(url):
    file_name = url.split('/')[-1]
    return re.sub(r'[:]', '_', file_name)

for url in tqdm(urls, desc="Descargando las curvas de luz"):
    file_name = clean_filename(url)
    file_path = os.path.join(lc_dir, file_name)
    if not os.path.exists(file_path):
        try:
            response = requests.get(url, stream=True)
            response.raise_for_status()
            with open(file_path, 'wb') as f:
                for chunk in response.iter_content(chunk_size=8192):
                    f.write(chunk)
            print(f"Descargado: {file_name}")
        except Exception as e:
            print(f"Error descargando {url}: {e}")
    else:
        print(f"Omitido, ya existe: {file_name}")

print("PRoceso finalizado")

Ahora realizo un preprocesamiento básico, normalizar, eliminar vacios y outlier extremadamente grandes que no se deberían a tránstios planetarios si no a otros motivos exógenos como errores de medición... Además, nos quedamos con el tipo de flujo pdcsap que ya está tratado para que podamos comparar las diferentes curvas de luz de las diferentes estrellas. Guardamos todo en un fichero pickle para usar más tarde.

In [ ]:
import numpy as np
import pandas as pd
from astropy.io import fits
from astropy.stats import sigma_clip
import pickle
import warnings

base_path = '/content/drive/My Drive/TFM_b'
lc_dir = os.path.join(base_path, 'light_curves_s68')
output_file = os.path.join(base_path, 'tess_sector_68_data.pkl')

def normalize_and_clean(time, flux, flux_err, quality, sigma=5):
    # Elimino NaN e infinitos
    valid = np.isfinite(time) & np.isfinite(flux) & np.isfinite(flux_err)
    time, flux, flux_err, quality = time[valid], flux[valid], flux_err[valid], quality[valid]

    if len(flux) == 0:
        return np.array([]), np.array([]), np.array([]), np.array([])

    # Normalizo el flujo
    median_flux = np.median(flux)
    if median_flux == 0:
        return np.array([]), np.array([]), np.array([]), np.array([])

    norm_flux = flux / median_flux
    norm_flux_err = flux_err / median_flux

    # Elimino outliers
    with warnings.catch_warnings():
        warnings.simplefilter("ignore")
        mask = sigma_clip(norm_flux, sigma=sigma, maxiters=5).mask

    return time[~mask], norm_flux[~mask], norm_flux_err[~mask], quality[~mask]

all_data = []

# Se procesa cada archivo FITS
for fits_file in tqdm(os.listdir(lc_dir)):
    if fits_file.endswith('.fits'):
        file_path = os.path.join(lc_dir, fits_file)

        try:
            with fits.open(file_path) as hdul:
                # Extraer TICID del header del archivo FITS
                ticid = hdul[0].header.get('TICID', None)
                if ticid is None:
                    print(f"No TICID en {fits_file}")
                    continue

                data = hdul[1].data
                header = hdul[0].header

                time = data['TIME']
                pdcsap_flux = data['PDCSAP_FLUX'] # USAMOS EL FLUJO PDCSAP!
                pdcsap_flux_err = data['PDCSAP_FLUX_ERR']
                quality = data['QUALITY']

                # Normaliza y limpia los datos
                time, norm_flux, norm_flux_err, quality = normalize_and_clean(time, pdcsap_flux, pdcsap_flux_err, quality)

                if len(time) > 0:
                    star_data = {
                        'ticid': ticid,
                        'time': time,
                        'norm_flux': norm_flux,
                        'norm_flux_err': norm_flux_err,
                        'quality': quality,
                        'tstart': header.get('TSTART', 0),
                        'tstop': header.get('TSTOP', 0),
                        'tessmag': header.get('TESSMAG', 0),
                        'teff': header.get('TEFF', 0),
                        'radius': header.get('RADIUS', 0)
                    }

                    all_data.append(star_data)
        except Exception as e:
            print(f"Error procesando {fits_file}: {str(e)}")

with open(output_file, 'wb') as f:
    pickle.dump(all_data, f)

print(f"Datos guardados en {output_file}")

Revisamos un poco el fichero y los datos que hemos descargado para comprobar su validez

In [ ]:
with open('/content/drive/My Drive/TFM_b/tess_sector_68_data.pkl', 'rb') as f:
    data = pickle.load(f)

print(f"Número total de objetos: {len(data)}")

unique_ticids = len(set(obj['ticid'] for obj in data if obj['ticid'] is not None))
print(f"Número de TICIDs únicos: {unique_ticids}")

print("\nInformación de los 10 primeros objetos:")
for i, obj in enumerate(data[:10]):
    print(f"\nObjeto {i+1}:")
    print(f"TICID: {obj['ticid']}")
    print(f"Número de puntos de datos: {len(obj['time'])}")
    print(f"Rango de tiempo de observación: {obj['tstart']:.2f} - {obj['tstop']:.2f}")
    print(f"Magnitud TESS: {obj['tessmag']:.2f}" if obj['tessmag'] is not None else "Magnitud TESS: No disponible")
    print(f"Temperatura efectiva: {obj['teff']:.2f} K" if obj['teff'] is not None else "Temperatura efectiva: No disponible")
    print(f"Radio estelar: {obj['radius']:.2f} R_sol" if obj['radius'] is not None else "Radio estelar: No disponible")


num_data_points = [len(obj['time']) for obj in data]
teff_values = [obj['teff'] for obj in data if obj['teff'] is not None]
radius_values = [obj['radius'] for obj in data if obj['radius'] is not None]
tessmag_values = [obj['tessmag'] for obj in data if obj['tessmag'] is not None]

print("\nEstadísticas generales:")
print(f"Promedio de puntos de datos por objeto: {np.mean(num_data_points):.2f}")
print(f"Mediana de puntos de datos por objeto: {np.median(num_data_points):.2f}")
if teff_values:
    print(f"Rango de temperaturas efectivas: {min(teff_values):.2f} - {max(teff_values):.2f} K")
else:
    print("No hay datos válidos de temperatura efectiva")
if radius_values:
    print(f"Rango de radios estelares: {min(radius_values):.2f} - {max(radius_values):.2f} R_sol")
else:
    print("No hay datos válidos de radio estelar")
if tessmag_values:
    print(f"Rango de magnitudes TESS: {min(tessmag_values):.2f} - {max(tessmag_values):.2f}")
else:
    print("No hay datos válidos de magnitud TESS")
    
# Ejemplo de una curva de luz
example_obj = data[0]
print(f"\nEjemplo de curva de luz para TICID {example_obj['ticid']}:")
print("Tiempo  Flujo Normalizado  Error de Flujo")
for t, f, e in zip(example_obj['time'][:5], example_obj['norm_flux'][:5], example_obj['norm_flux_err'][:5]):
    print(f"{t:.2f}  {f:.5f}  {e:.5f}")

Número total de objetos: 10573
Número de TICIDs únicos: 10573

Información de los 10 primeros objetos:

Objeto 1:
TICID: 139988455
Número de puntos de datos: 14957
Rango de tiempo de observación: 3154.62 - 3182.14
Magnitud TESS: 9.00
Temperatura efectiva: 4751.40 K
Radio estelar: 11.07 R_sol

Objeto 2:
TICID: 139990152
Número de puntos de datos: 14747
Rango de tiempo de observación: 3154.63 - 3182.13
Magnitud TESS: 11.50
Temperatura efectiva: 4640.85 K
Radio estelar: 0.76 R_sol

Objeto 3:
TICID: 139988632
Número de puntos de datos: 14956
Rango de tiempo de observación: 3154.63 - 3182.13
Magnitud TESS: 9.75
Temperatura efectiva: 5032.00 K
Radio estelar: 0.80 R_sol

Objeto 4:
TICID: 139990632
Número de puntos de datos: 14678
Rango de tiempo de observación: 3154.63 - 3182.13
Magnitud TESS: 14.27
Temperatura efectiva: 3037.00 K
Radio estelar: 0.18 R_sol

Objeto 5:
TICID: 139990402
Número de puntos de datos: 14956
Rango de tiempo de observación: 3154.63 - 3182.13
Magnitud TESS: 9.69
Temperatura efectiva: 4853.00 K
Radio estelar: 0.84 R_sol

Objeto 6:
TICID: 139993838
Número de puntos de datos: 14957
Rango de tiempo de observación: 3154.63 - 3182.13
Magnitud TESS: 9.60
Temperatura efectiva: 4410.00 K
Radio estelar: 2.98 R_sol

Objeto 7:
TICID: 139994232
Número de puntos de datos: 14896
Rango de tiempo de observación: 3154.62 - 3182.14
Magnitud TESS: 9.75
Temperatura efectiva: 4297.50 K
Radio estelar: 24.59 R_sol

Objeto 8:
TICID: 139993199
Número de puntos de datos: 14931
Rango de tiempo de observación: 3154.63 - 3182.13
Magnitud TESS: 12.48
Temperatura efectiva: 3141.00 K
Radio estelar: 0.21 R_sol

Objeto 9:
TICID: 139994847
Número de puntos de datos: 14956
Rango de tiempo de observación: 3154.63 - 3182.13
Magnitud TESS: 7.76
Temperatura efectiva: 6187.97 K
Radio estelar: 2.45 R_sol

Objeto 10:
TICID: 139995595
Número de puntos de datos: 14957
Rango de tiempo de observación: 3154.62 - 3182.14
Magnitud TESS: 8.90
Temperatura efectiva: 4698.10 K
Radio estelar: 10.19 R_sol

Estadísticas generales:
Promedio de puntos de datos por objeto: 14855.21
Mediana de puntos de datos por objeto: 14957.00
Rango de temperaturas efectivas: 2768.00 - 52460.00 K
Rango de radios estelares: 0.09 - 198.82 R_sol
Rango de magnitudes TESS: 1.71 - 19.84

Ejemplo de curva de luz para TICID 139988455:
Tiempo  Flujo Normalizado  Error de Flujo
3154.88  1.00053  0.00062
3154.88  0.99947  0.00062
3154.89  1.00052  0.00062
3154.89  0.99953  0.00062
3154.89  0.99985  0.00062

___________________________________________________________

Revisar si hay planetas o candidatos en este dataset. Este dataset del sector 68 me sirve para popular el dataset con ejemplos de exoplaneta = 0, por lo que en caso de haber planetas deberíamos eliminarlos de este dataset. En caso de haber candidatos, vamos a guardarlos en un dataset a parte para luego chequear nuestro futuro modelo sobre ellos.

In [ ]:
from collections import Counter

with open('/content/drive/My Drive/TFM_b/tess_sector_68_data_updated.pkl', 'rb') as f:
    data = pickle.load(f)

candidates = []
non_exoplanets = []

for obj in data:
    if obj['exoplanet'] == 2:
        candidates.append(obj)
    elif obj['exoplanet'] == 0:
        non_exoplanets.append(obj)

# Fichero para los CANDIDATOS
candidates_file = '/content/drive/My Drive/TFM_b/tess_sector_68_candidates.pkl'
with open(candidates_file, 'wb') as f:
    pickle.dump(candidates, f)

# Fichero para EL SECTOR 68 SIN EXOPLANETAS
non_exoplanets_file = '/content/drive/My Drive/TFM_b/tess_sector_68_non_exoplanets.pkl'
with open(non_exoplanets_file, 'wb') as f:
    pickle.dump(non_exoplanets, f)

print(f"Total de objetos: {len(data)}")
print(f"Candidatos a exoplanetas: {len(candidates)}")
print(f"No exoplanetas: {len(non_exoplanets)}")

print(f"\nDatos de candidatos guardados en: {candidates_file}")
print(f"Datos de no exoplanetas guardados en: {non_exoplanets_file}")

def check_distribution(dataset):
    counter = Counter(obj['exoplanet'] for obj in dataset)
    return counter

print("\nDistribución en el conjunto de candidatos:")
print(check_distribution(candidates))

print("\nDistribución en el conjunto de no exoplanetas:")
print(check_distribution(non_exoplanets))

def show_examples(dataset, name, n=5):
    print(f"\nEjemplos de {name}:")
    for i, obj in enumerate(dataset[:n]):
        print(f"Objeto {i+1}:")
        print(f"  TICID: {obj['ticid']}")
        print(f"  Exoplanet status: {obj['exoplanet']}")
        print(f"  Número de puntos de datos: {len(obj['time'])}")
        print(f"  Magnitud TESS: {obj['tessmag']:.2f}")
        print(f"  Temperatura efectiva: {obj['teff']:.2f} K")
        print(f"  Radio estelar: {obj['radius']:.2f} R_sol")
        print()

show_examples(candidates, "candidatos")
show_examples(non_exoplanets, "no exoplanetas")

Total de objetos: 10573
Candidatos a exoplanetas: 400
No exoplanetas: 10173

Datos de candidatos guardados en: /content/drive/My Drive/TFM_b/tess_sector_68_candidates.pkl
Datos de no exoplanetas guardados en: /content/drive/My Drive/TFM_b/tess_sector_68_non_exoplanets.pkl

Distribución en el conjunto de candidatos:
Counter({2: 400})

Distribución en el conjunto de no exoplanetas:
Counter({0: 10173})

Ejemplos de candidatos:
Objeto 1:
  TICID: 140067837
  Exoplanet status: 2
  Número de puntos de datos: 14800
  Magnitud TESS: 12.45
  Temperatura efectiva: 5734.00 K
  Radio estelar: 1.08 R_sol

Objeto 2:
  TICID: 140068425
  Exoplanet status: 2
  Número de puntos de datos: 14953
  Magnitud TESS: 9.78
  Temperatura efectiva: 6231.00 K
  Radio estelar: 2.36 R_sol

Objeto 3:
  TICID: 140687214
  Exoplanet status: 2
  Número de puntos de datos: 15064
  Magnitud TESS: 12.13
  Temperatura efectiva: 3672.00 K
  Radio estelar: 0.57 R_sol

Objeto 4:
  TICID: 140830390
  Exoplanet status: 2
  Número de puntos de datos: 15121
  Magnitud TESS: 11.71
  Temperatura efectiva: 5815.00 K
  Radio estelar: 2.45 R_sol

Objeto 5:
  TICID: 140893722
  Exoplanet status: 2
  Número de puntos de datos: 15106
  Magnitud TESS: 11.59
  Temperatura efectiva: 4468.00 K
  Radio estelar: 0.74 R_sol


Ejemplos de no exoplanetas:
Objeto 1:
  TICID: 139988455
  Exoplanet status: 0
  Número de puntos de datos: 14957
  Magnitud TESS: 9.00
  Temperatura efectiva: 4751.40 K
  Radio estelar: 11.07 R_sol

Objeto 2:
  TICID: 139990152
  Exoplanet status: 0
  Número de puntos de datos: 14747
  Magnitud TESS: 11.50
  Temperatura efectiva: 4640.85 K
  Radio estelar: 0.76 R_sol

Objeto 3:
  TICID: 139988632
  Exoplanet status: 0
  Número de puntos de datos: 14956
  Magnitud TESS: 9.75
  Temperatura efectiva: 5032.00 K
  Radio estelar: 0.80 R_sol

Objeto 4:
  TICID: 139990632
  Exoplanet status: 0
  Número de puntos de datos: 14678
  Magnitud TESS: 14.27
  Temperatura efectiva: 3037.00 K
  Radio estelar: 0.18 R_sol

Objeto 5:
  TICID: 139990402
  Exoplanet status: 0
  Número de puntos de datos: 14956
  Magnitud TESS: 9.69
  Temperatura efectiva: 4853.00 K
  Radio estelar: 0.84 R_sol